# Import


In [35]:
# !pip install ipympl
# !pip3 install roboticstoolbox-python
%matplotlib widget
# PyTorch
import torch
import torch.nn as nn
from torch.optim import optimizer

# For data preprocess
import numpy as np
import csv
import os

# For plotting
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

from roboticstoolbox import DHRobot, RevoluteMDH 
import math

# GPU infomation


In [36]:
GPU_name = torch.cuda.get_device_name()
print(GPU_name)

NVIDIA GeForce GTX 1050


# Utilities

In [37]:
def get_device():
    ''' Get device (if GPU is available, use GPU) '''
    return 'cuda' if torch.cuda.is_available() else 'cpu'

def plot_learning_curve(loss_record, title=''):
    ''' Plot learning curve of your DNN (train & dev loss) '''
    total_steps = len(loss_record['train'])
    x_1 = range(total_steps)
    # x_2 = x_1[::len(loss_record['train']) // len(loss_record['dev'])]
    x_2 = range(total_steps)
    figure(figsize=(6, 4))
    plt.plot(x_1, loss_record['train'], c='tab:red', label='train')
    plt.plot(x_2, loss_record['dev'], c='tab:cyan', label='dev')
    plt.ylim(0.0, 1.)
    plt.xlabel('Training steps')
    plt.ylabel('MSE loss')
    plt.title('Learning curve of {}'.format(title))
    plt.legend()
    plt.show()

# Network


In [38]:
class NeuralNet(nn.Module):
    ''' A neural network '''
    def __init__(self, input_dim, output_dim):
        super(NeuralNet, self).__init__()

        # Define your neural network here
        # TODO: How to modify this model to achieve better performance?
        self.net = nn.Sequential(
            nn.Linear(input_dim, 32),
            nn.Sigmoid(),
            nn.Linear(32, 64),
            nn.RReLU(),
            nn.Linear(64, 32),
            nn.LeakyReLU(),
            nn.Linear(32, output_dim)
        )

        # Mean squared error loss
        self.criterion = nn.MSELoss(reduction='mean')

    def forward(self, x):
        ''' Given input of size (batch_size x input_dim), compute output of the network '''
        return self.net(x)

    def cal_loss(self, pred, target):
        ''' Calculate loss '''
        # def euclidean_distance(y_true, y_pred):
        # return kr.sqrt(kr.sum(kr.square(y_true - y_pred)))
        return self.criterion(pred, target)

# 宣告Dobot位置與軸關節公式函數(順向運動學)


In [39]:
def dobot_forword_kine(joints):

    if joints.ndim == 1:
        joints = np.expand_dims(joints, 0)

    q1 = joints[:, 0:1] # 0:1而非直接0->確保內部採用Column處理
    q2 = joints[:, 1:2]
    q3 = joints[:, 2:3]

    a2 = 135
    a3 = 147
    a4 = 61

    C1 = np.cos(q1)
    C2 = np.cos(q2)
    C23 = np.cos(q2 + q3)
    S1 = np.sin(q1)
    S2 = np.sin(q2)
    S23 = np.sin(q2 + q3)

    dx = C1 * (a3 * C23 + a2 * C2 + a4)
    dy = S1 * (a3 * C23 + a2 * C2 + a4)
    dz = -a2 * S2 - a3 * S23	

    Point = np.hstack([dx, dy, dz]) # 建立陣列
    return Point

# 產生訓練集資料

In [40]:
def gen_data(Train_num):
    joint_1 = (-np.pi / 2) + np.pi * np.random.rand(Train_num, 1)
    joint_2 = (-85 * np.pi / 180) + (85 * np.pi / 180) * np.random.rand(Train_num, 1)
    joint_3 = (-10 * np.pi / 180) + (105 * np.pi / 180) * np.random.rand(Train_num, 1)
    joints = np.hstack((joint_1, joint_2, joint_3))
    points = dobot_forword_kine(joints)
    return points, joints

In [41]:
x_train, y_train = gen_data(10000)
x_dev, y_dev = gen_data(500)

# Dev Func


In [42]:
def dev(feature_dev,target_dev, model, device):
    model.eval()                                # set model to evalutation mode
    with torch.no_grad():                   # disable gradient calculation
        pred = model(feature_dev)                     # forward pass (compute output)
        mse_loss = model.cal_loss(pred, target_dev)  # compute loss
    return mse_loss

# Train Setup

In [43]:
# Ref: https://ithelp.ithome.com.tw/articles/10276281

# 0) prepare data

feature = torch.from_numpy(x_train)
print(feature.size())
target = torch.from_numpy(y_train)
print(target.size())

feature_dev = torch.from_numpy(x_dev)
target_dev = torch.from_numpy(y_dev)

feature,target=feature.type(torch.FloatTensor),target.type(torch.FloatTensor)
feature_dev,target_dev=feature_dev.type(torch.FloatTensor),target_dev.type(torch.FloatTensor)


n_samples, n_features = feature.shape
print(n_samples)
print(n_features)

# 1) model
model = NeuralNet(n_features, n_features)
# 2) loss and optimizer
learning_rate = 0.001
# optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# 3) training loop
epochs = 10000
min_mse = 1000.0
os.makedirs('models', exist_ok=True)
device = get_device()                 # get the current available device ('cpu' or 'cuda')
loss_record = {'train': [], 'dev': []} 

torch.Size([10000, 3])
torch.Size([10000, 3])
10000
3


# Training Loop

In [44]:
for epoch in range(epochs):
    model.train()
    # forward pass and loss
    y_predicted = model(feature)
    # print(y_predicted)
    # loss = criterion(y_predicted, target)
    mse_loss = model.cal_loss(y_predicted, target)

    # backward pass
    mse_loss.backward()


    # update
    optimizer.step()
    loss_record['train'].append(mse_loss.detach().cpu().item())

    dev_mse = dev(feature_dev,target_dev, model, device)
    loss_record['dev'].append(dev_mse)

    # init optimizer
    optimizer.zero_grad()

    if (epoch + 1) % 10 == 0:
        print(f'epoch: {epoch+1}, dev_loss = {dev_mse.item(): .4f}')
        if (dev_mse < min_mse):
            torch.save(model.state_dict(), 'models/model.pth')
            min_mse = dev_mse
            print('Saving model (epoch = {:4d}, train_loss = {:.4f}, dev_loss = {:.4f})'
                    .format(epoch + 1, mse_loss, dev_mse))
            

print('Finished training after {} epochs'.format(epoch+1))

epoch: 10, dev_loss =  0.4995
Saving model (epoch =   10, train_loss = 0.5196, dev_loss = 0.4995)
epoch: 20, dev_loss =  0.3492
Saving model (epoch =   20, train_loss = 0.3479, dev_loss = 0.3492)
epoch: 30, dev_loss =  0.2456
Saving model (epoch =   30, train_loss = 0.2494, dev_loss = 0.2456)
epoch: 40, dev_loss =  0.1611
Saving model (epoch =   40, train_loss = 0.1647, dev_loss = 0.1611)
epoch: 50, dev_loss =  0.1271
Saving model (epoch =   50, train_loss = 0.1279, dev_loss = 0.1271)
epoch: 60, dev_loss =  0.1143
Saving model (epoch =   60, train_loss = 0.1134, dev_loss = 0.1143)
epoch: 70, dev_loss =  0.1034
Saving model (epoch =   70, train_loss = 0.1037, dev_loss = 0.1034)
epoch: 80, dev_loss =  0.0990
Saving model (epoch =   80, train_loss = 0.0987, dev_loss = 0.0990)
epoch: 90, dev_loss =  0.0964
Saving model (epoch =   90, train_loss = 0.0961, dev_loss = 0.0964)
epoch: 100, dev_loss =  0.0944
Saving model (epoch =  100, train_loss = 0.0943, dev_loss = 0.0944)
epoch: 110, dev_los

# Test

In [45]:
# generate test data
# in this problem, the test set is not completely unknown, so I print it out and even calculate the loss
x_test, y_test = gen_data(20)
test_feature = torch.from_numpy(x_test)
test_target = torch.from_numpy(y_test)
test_feature,test_target=test_feature.type(torch.FloatTensor),test_target.type(torch.FloatTensor)
n_samples, n_features = test_feature.shape

In [46]:
del model
model = NeuralNet(n_features, n_features).to(device)
ckpt = torch.load('models/model.pth', map_location='cuda:0')  # Load your best model
model.load_state_dict(ckpt)

<All keys matched successfully>

In [47]:
model.eval()   
test_feature = test_feature.to(device) 
test_target = test_target.to(device)                             # set model to evalutation mode
with torch.no_grad():                   # disable gradient calculation
    pred = model(test_feature)                     # forward pass (compute output)
# print("test_feature: ")
# print(test_feature)
print("y_pred: ")
print(pred)
print("y_test: ")
print(test_target)
print("test MSE loss: ")
test_loss = model.cal_loss(pred, test_target)
print(test_loss)

y_pred: 
tensor([[ 1.0309, -0.2603,  1.1671],
        [ 0.6087, -0.9273,  1.3587],
        [-0.6182, -0.4058,  0.1412],
        [-0.3937, -0.4154,  0.9144],
        [ 0.9827, -0.3994,  0.1601],
        [-0.5744, -0.7414,  1.0962],
        [-1.5178, -0.4404,  0.2608],
        [ 0.6324, -0.3535,  0.1574],
        [ 0.7522, -1.2607,  0.1104],
        [-0.5909, -0.6213,  1.0871],
        [-1.5148, -0.6689,  0.9735],
        [ 0.4717, -0.4946,  1.0710],
        [-0.2880, -0.7859,  0.1312],
        [ 0.2714, -0.5477,  1.0575],
        [-0.2078, -0.4274,  0.1446],
        [ 0.6816, -0.0065,  0.9519],
        [ 0.6133, -1.0259,  0.9669],
        [ 1.2654, -0.3339,  1.0582],
        [-0.9976, -0.6483,  0.2607],
        [ 0.0508, -0.4525,  0.1292]], device='cuda:0')
y_test: 
tensor([[ 1.0620, -0.2163,  1.0660],
        [ 0.6289, -0.8834,  1.2974],
        [-0.6134, -0.4037,  0.1619],
        [-0.3873, -0.4349,  0.9682],
        [ 1.0044, -0.3704,  0.1104],
        [-0.5606, -0.7559,  1.1477],
  

# Visualization

In [48]:
plot_learning_curve(loss_record, title='deep model')
# the learning curve of dev and train overlaps since this problem is simple

FigureCanvasNbAgg()

# Robot Arm

In [49]:
# DH ref: https://forum.dobot.cc/t/dobot-magicain-kinematics/3305
# Scale down 1/1000 to fit in the plot
class DOBOT(DHRobot):
    def __init__(self):
        super().__init__(
                [
                    RevoluteMDH(),
                    RevoluteMDH(alpha=np.pi/2),
                    RevoluteMDH(a=0.135),
                    RevoluteMDH(a=0.147),
                    RevoluteMDH(a=0.061),
                ], name="DOBOT")        

In [50]:
robot = DOBOT()
print(robot)
test = [0 ,0 ,0, 0, 0]
robot.plot(test)


DHRobot: DOBOT, 5 joints (RRRRR), dynamics, modified DH parameters
┏━━━━━━┳━━━━━━━┳━━━━━┳━━━━━┓
┃aⱼ₋₁  ┃ ⍺ⱼ₋₁  ┃ θⱼ  ┃ dⱼ  ┃
┣━━━━━━╋━━━━━━━╋━━━━━╋━━━━━┫
┃  0.0 ┃  0.0° ┃  q1 ┃ 0.0 ┃
┃  0.0 ┃ 90.0° ┃  q2 ┃ 0.0 ┃
┃0.135 ┃  0.0° ┃  q3 ┃ 0.0 ┃
┃0.147 ┃  0.0° ┃  q4 ┃ 0.0 ┃
┃0.061 ┃  0.0° ┃  q5 ┃ 0.0 ┃
┗━━━━━━┻━━━━━━━┻━━━━━┻━━━━━┛



FigureCanvasNbAgg()

PyPlot3D backend, t = 0.05, scene:
  DOBOT

In [51]:
arr = pred[1].cpu().detach().numpy()
if(abs(arr[2])>abs(arr[1])):
    t4 = np.sign(arr[1]) * (abs(arr[2])-abs(arr[1]))
else:
    t4 =  np.sign(arr[2]) * (abs(arr[1])-abs(arr[2]))

print('pred')
print(arr)
print('')

print('target')
print(test_target[1])
print('')

arr = np.append(arr, np.array([t4,0]), axis=0)

print('joint config')
print(arr)
print('')

T = robot.fkine(arr)
print(T)
# sol = robot.ikine_LM(T)
# print(sol)
# print('')
print("X Y Z")
print(test_feature[1])
print('')
robot.plot(arr)

pred
[ 0.6087154 -0.9272763  1.3586692]

target
tensor([ 0.6289, -0.8834,  1.2974], device='cuda:0')

joint config
[ 0.60871542 -0.92727631  1.35866916 -0.43139285  0.        ]

   0.8204    0         0.5718    0.226     
   0.5718    0        -0.8204    0.1576    
   0         1         0        -0.04653   
   0         0         0         1         

X Y Z
tensor([227.4335, 165.4449,  45.2012], device='cuda:0')



PyPlot3D backend, t = 0.05, scene:
  DOBOT